In this assignment, students will be using the K-nearest neighbors
algorithm to predict how many points NBA players scored in the 2013-2014
season.

A look at the data

Before we dive into the algorithm, let’s take a look at our data. Each row in
the data contains information on how a player performed in the 2013-2014
NBA season.

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
from sklearn.model_selection import KFold

In [2]:
df=pd.read_csv('nba_2013.csv')

In [3]:
df.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   player        481 non-null    object 
 1   pos           481 non-null    object 
 2   age           481 non-null    int64  
 3   bref_team_id  481 non-null    object 
 4   g             481 non-null    int64  
 5   gs            481 non-null    int64  
 6   mp            481 non-null    int64  
 7   fg            481 non-null    int64  
 8   fga           481 non-null    int64  
 9   fg.           479 non-null    float64
 10  x3p           481 non-null    int64  
 11  x3pa          481 non-null    int64  
 12  x3p.          414 non-null    float64
 13  x2p           481 non-null    int64  
 14  x2pa          481 non-null    int64  
 15  x2p.          478 non-null    float64
 16  efg.          479 non-null    float64
 17  ft            481 non-null    int64  
 18  fta           481 non-null    

In [5]:
df.describe()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,season_end
count,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,479.000000,481.000000,481.000000,414.000000,...,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.0
mean,26.509356,53.253638,25.571726,1237.386694,192.881497,424.463617,0.436436,39.613306,110.130977,0.285111,...,55.810811,162.817048,218.627859,112.536383,39.280665,24.103950,71.862786,105.869023,516.582121,2013.0
std,4.198265,25.322711,29.658465,897.258840,171.832793,368.850833,0.098672,50.855639,132.751732,0.157633,...,62.101191,145.348116,200.356507,131.019557,34.783590,30.875381,62.701690,71.213627,470.422228,0.0
min,19.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.0
25%,23.000000,32.000000,0.000000,388.000000,47.000000,110.000000,0.400500,0.000000,3.000000,0.234355,...,12.000000,43.000000,55.000000,20.000000,9.000000,4.000000,21.000000,44.000000,115.000000,2013.0
50%,26.000000,61.000000,10.000000,1141.000000,146.000000,332.000000,0.438000,16.000000,48.000000,0.330976,...,35.000000,135.000000,168.000000,65.000000,32.000000,14.000000,58.000000,104.000000,401.000000,2013.0
75%,29.000000,76.000000,54.000000,2016.000000,307.000000,672.000000,0.479500,68.000000,193.000000,0.375000,...,73.000000,230.000000,310.000000,152.000000,60.000000,32.000000,108.000000,158.000000,821.000000,2013.0
max,39.000000,83.000000,82.000000,3122.000000,849.000000,1688.000000,1.000000,261.000000,615.000000,1.000000,...,440.000000,783.000000,1114.000000,721.000000,191.000000,219.000000,295.000000,273.000000,2593.000000,2013.0


In [6]:
df.isna().sum()

player           0
pos              0
age              0
bref_team_id     0
g                0
gs               0
mp               0
fg               0
fga              0
fg.              2
x3p              0
x3pa             0
x3p.            67
x2p              0
x2pa             0
x2p.             3
efg.             2
ft               0
fta              0
ft.             20
orb              0
drb              0
trb              0
ast              0
stl              0
blk              0
tov              0
pf               0
pts              0
season           0
season_end       0
dtype: int64

In [8]:
df["fg."].fillna(df["fg."].mean(),inplace=True)
df["x2p."].fillna(df["x2p."].mean(),inplace=True)
df["efg."].fillna(df["efg."].mean(),inplace=True)
df["x3p."].fillna(df["x3p."].mean(),inplace=True)
df["ft."].fillna(df["ft."].mean(),inplace=True)

In [30]:
l=[]
for i in df.columns:
    if not df[i].dtypes==object:
        l.append(i)

In [31]:
l

['age',
 'g',
 'gs',
 'mp',
 'fg',
 'fga',
 'fg.',
 'x3p',
 'x3pa',
 'x3p.',
 'x2p',
 'x2pa',
 'x2p.',
 'efg.',
 'ft',
 'fta',
 'ft.',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'season_end']

In [32]:
df_numeric=df[l]
df_numeric.head()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,season_end
0,23,63,0,847,66,141,0.468,4,15,0.266667,...,72,144,216,28,23,26,30,122,171,2013
1,20,81,20,1197,93,185,0.503,0,0,0.285111,...,142,190,332,43,40,57,71,203,265,2013
2,27,53,12,961,143,275,0.520,0,0,0.285111,...,102,204,306,38,24,36,39,108,362,2013
3,28,73,73,2552,464,1011,0.459,128,300,0.426667,...,32,230,262,248,35,3,146,136,1330,2013
4,25,56,30,951,136,249,0.546,0,1,0.000000,...,94,183,277,40,23,46,63,187,328,2013


In [67]:
min_max=MinMaxScaler()
min_max_scaled=min_max.fit_transform(df_numeric)
df_minmax=pd.DataFrame(min_max_scaled,columns=df_numeric.columns)

In [68]:
df_minmax.head()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,season_end
0,0.20,0.756098,0.000000,0.271067,0.077739,0.083531,0.468,0.015326,0.024390,0.266667,...,0.163636,0.183908,0.193896,0.038835,0.120419,0.118721,0.101695,0.446886,0.065947,0.0
1,0.05,0.975610,0.243902,0.383211,0.109541,0.109597,0.503,0.000000,0.000000,0.285111,...,0.322727,0.242656,0.298025,0.059639,0.209424,0.260274,0.240678,0.743590,0.102198,0.0
2,0.40,0.634146,0.146341,0.307594,0.168433,0.162915,0.520,0.000000,0.000000,0.285111,...,0.231818,0.260536,0.274686,0.052705,0.125654,0.164384,0.132203,0.395604,0.139607,0.0
3,0.45,0.878049,0.890244,0.817366,0.546525,0.598934,0.459,0.490421,0.487805,0.426667,...,0.072727,0.293742,0.235189,0.343967,0.183246,0.013699,0.494915,0.498168,0.512919,0.0
4,0.30,0.670732,0.365854,0.304390,0.160188,0.147512,0.546,0.000000,0.001626,0.000000,...,0.213636,0.233716,0.248654,0.055479,0.120419,0.210046,0.213559,0.684982,0.126494,0.0


In [42]:
df_categorical=df[['player','bref_team_id','season']]

In [44]:
nba=pd.concat([df_categorical,df_minmax], axis=1)

In [45]:
nba.head()

,player,bref_team_id,season,age,g,gs,mp,fg,fga,fg.,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,season_end
0,Quincy Acy,TOT,2013-2014,0.20,0.756098,0.000000,0.271067,0.077739,0.083531,0.468,...,0.163636,0.183908,0.193896,0.038835,0.120419,0.118721,0.101695,0.446886,0.065947,0.0
1,Steven Adams,OKC,2013-2014,0.05,0.975610,0.243902,0.383211,0.109541,0.109597,0.503,...,0.322727,0.242656,0.298025,0.059639,0.209424,0.260274,0.240678,0.743590,0.102198,0.0
2,Jeff Adrien,TOT,2013-2014,0.40,0.634146,0.146341,0.307594,0.168433,0.162915,0.520,...,0.231818,0.260536,0.274686,0.052705,0.125654,0.164384,0.132203,0.395604,0.139607,0.0
3,Arron Afflalo,ORL,2013-2014,0.45,0.878049,0.890244,0.817366,0.546525,0.598934,0.459,...,0.072727,0.293742,0.235189,0.343967,0.183246,0.013699,0.494915,0.498168,0.512919,0.0
4,Alexis Ajinca,NOP,2013-2014,0.30,0.670732,0.365854,0.304390,0.160188,0.147512,0.546,...,0.213636,0.233716,0.248654,0.055479,0.120419,0.210046,0.213559,0.684982,0.126494,0.0


In [47]:
len(nba.columns)

30

In [58]:
x=nba.drop(['player','bref_team_id','season','pts','season_end'],axis=1)
y=nba['pts']

In [71]:
y

0      0.065947
1      0.102198
2      0.139607
3      0.512919
4      0.126494
         ...   
476    0.362129
477    0.441188
478    0.546471
479    0.188970
480    0.153876
Name: pts, Length: 481, dtype: float64

In [60]:
len(x.columns)

25

In [87]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=110)

In [88]:
for i in range(10):
    k=i+1
    knn=KNeighborsRegressor(n_neighbors=k)
    knn.fit(x_train,y_train) 
    y_pred=knn.predict(x_test)
    print("r2 score is:",format(metrics.r2_score(y_test, y_pred),'.3f'), "for k =", k)

r2 score is: 0.939 for k = 1
r2 score is: 0.954 for k = 2
r2 score is: 0.965 for k = 3
r2 score is: 0.971 for k = 4
r2 score is: 0.975 for k = 5
r2 score is: 0.972 for k = 6
r2 score is: 0.972 for k = 7
r2 score is: 0.969 for k = 8
r2 score is: 0.966 for k = 9
r2 score is: 0.965 for k = 10


In [91]:
knn=KNeighborsRegressor(n_neighbors = 5)
knn.fit(x_train, y_train) 
y_pred=knn.predict(x_test)
print("Mean Squared Error is:",metrics.mean_squared_error(y_test, y_pred))
print("Regression score is:",metrics.r2_score(y_test, y_pred))

Mean Squared Error is: 0.0007338523348320798
Regression score is: 0.9750852552496283


In [94]:
print("Accuracy of training data :",knn.score(x_train,y_train))

Accuracy of training data : 0.9721708941813321


In [95]:
print("Accuracy of testing data :",knn.score(x_test,y_test))

Accuracy of testing data : 0.9750852552496283


In [96]:
y_pred

array([8.24527574e-02, 2.70497493e-01, 4.34323178e-01, 5.09834169e-02,
       1.26494408e-02, 7.49710760e-02, 3.37292711e-01, 3.02506749e-01,
       8.94716545e-03, 3.63979946e-01, 3.16390282e-01, 4.12572310e-01,
       5.31045121e-01, 7.48168145e-02, 3.70921712e-01, 1.39992287e-01,
       1.19398380e-01, 3.85576552e-01, 2.74971076e-01, 3.30813729e-01,
       7.83956807e-01, 8.11415349e-02, 6.39413806e-02, 1.06440417e-02,
       2.59544929e-01, 3.42306209e-01, 9.10142692e-03, 1.27574238e-01,
       2.01774007e-01, 4.70266101e-01, 4.76667952e-02, 9.40994987e-02,
       2.04396452e-01, 2.71500193e-02, 1.33436174e-02, 2.68260702e-01,
       1.80485924e-02, 3.95912071e-01, 3.67913614e-02, 2.89625916e-01,
       2.26764366e-02, 4.73968376e-01, 1.41226379e-01, 3.11299653e-01,
       2.32009256e-01, 3.58657925e-02, 5.29811030e-01, 4.87697648e-01,
       1.22329348e-01, 1.16390282e-01, 1.63054377e-01, 1.31122252e-02,
       2.01002700e-01, 1.60046278e-01, 3.80100270e-01, 2.88468955e-02,
      